<h1 id="intro">Text Tokenization - Three Data Splitting Approaches with a Chat Template</h1>

This notebook provides a walkthrough of using [chat templates](https://huggingface.co/docs/transformers/main/en/chat_templating) and three text splitting approaches. The example in this notebook is identical to the [tokenizer-three-approaches.ipynb](./tokenizer-three-approaches.ipynb) **EXCEPT** this notebook **INCLUDES** a `chat template` which requires additional steps. The concepts in this notebook can be used to gain familiarity with tokenizers which are fundamental for NLP.

## Table of Contents
If viewing this notebook from GitHub please view it instead on [nbviewer.org](https://nbviewer.org/) so the hyperlinks will function. 

- [User Inputs](#user-inputs)
- [Import Libraries and Modules](#import-libs)
- [Tokenizer](#tokenizer)
- [Chat Template](#chat-template)
- [Approach 1: Right Side Truncation](#approach-1-right-side-truncation)
- [Approach 2: Right and Left Side of Text](#approach-2-right-left-side-truncation)
- [Approach 3: Chunking with Overlap](#approach-3-chunking-overlap)
- [Takeaways](#takeaways)

<h1 id="user-inputs">User Inputs</h1>

##### [Return To Top](#intro)

In [1]:
# Model name
model_name = 'gemma-2-9b-it'

# Tokenizer max length and stride
max_length = 8
stride = 1

<h1 id="import-libs">Import Libraries and Modules</h1>

##### [Return To Top](#intro)

In [2]:
# Libraries
import os
import sys
from pathlib import Path
import gc
from omegaconf import OmegaConf
from omegaconf.dictconfig import DictConfig
import polars as pl
import numpy as np
from transformers import AutoTokenizer
import torch

# Setup HF env. variables
os.environ['TRANSFORMERS_OFFLINE'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'True'
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [3]:
# Sample text for this notebook
text = (f'This is text that will be chunked '
        f'with overlap for simple example use cases')

<h1 id="tokenizer">Tokenizer</h1>

The Gemma 2 model uses a [SentencePiece tokenizer](https://arxiv.org/html/2408.00118v1) with a vocabulary size of 256K.

##### [Return To Top](#intro)

In [4]:
# Path to the model and tokenizer model card saved on disk
model_path = Path(os.getenv('LLM_MODELS')) / model_name

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Number of tokens in the tokenizer
print(f'Number of Token in Vocabulary: {len(tokenizer.get_vocab()):,}')

Number of Token in Vocabulary: 256,000


Special tokens in Hugging Face tokenizers are used to indicate the start and end of a sequence, separate sentences, or represent padding and unknown tokens. These tokens help the model understand the structure and boundaries of the input text during training and inference.

In [5]:
# Special tokens
print('Special Tokens')
for token_name in tokenizer.SPECIAL_TOKENS_ATTRIBUTES:
    token = getattr(tokenizer, token_name)
    if token_name == 'additional_special_tokens':
        token_id = None
    else:
        token_id = getattr(tokenizer, f'{token_name}_id')
    print(f'{token_name}: {token} -> {token_id}')

# Additional special tokens
print(f'\nAdditional Special Tokens')
for token_id in tokenizer.additional_special_tokens_ids:
    print(f'{token_id}: {tokenizer.decode(token_id)}')

Special Tokens
bos_token: <bos> -> 2
eos_token: <eos> -> 1
unk_token: <unk> -> 3
sep_token: None -> None
pad_token: <pad> -> 0
cls_token: None -> None
mask_token: None -> None
additional_special_tokens: ['<start_of_turn>', '<end_of_turn>'] -> None

Additional Special Tokens
106: <start_of_turn>
107: <end_of_turn>


In [6]:
from typing import Type, TypeVar

T = TypeVar('T')

def view_tokens(tok: Type[T], input_ids: list) -> None:
    """Decode and Print input_ids tokens for viewing 

    Args:
        tok (Type[T]): Tokenizer
        input_ids (list): List of input_id tokens
    """
    print(f'input_ids:\n{input_ids}\n\n')
    print(f'tokenizer.convert_ids_to_tokens(input_ids):\n'
          f'{tok.convert_ids_to_tokens(input_ids)}\n\n')
    
    print(f'tokenizer.decode(input_ids):\n'
          f'{tok.decode(input_ids)}\n')
    return

<h1 id="chat-template">Chat Template</h1>

The chat template in Hugging Face models is used to structure and format the input text for conversational AI models. It defines how the user and assistant inputs should be formatted, including prefixes, separators, and special tokens, ensuring that the model can properly differentiate between user prompts and assistant responses. This template helps maintain the context and flow of the conversation during training and inference.

##### [Return To Top](#intro)

In [7]:
# Chat template
empty_text = ''
message = [{'role': 'user', 'content': empty_text}]
tk = tokenizer.apply_chat_template(conversation=message,
                                   tokenize=True,
                                   add_generation_prompt=False,
                                   return_dict=True,
                                   tokenizer_kwargs={'return_length': True}
                                   )

# Number of tokens in the chat template
num_token_chat_template = tk['length'][0]

print(f'len(tk): {len(tk)}\n')
print(f'tk.keys(): {tk.keys()}\n')
print(f'Number of Tokens: {tk["length"][0]}\n')
view_tokens(tok=tokenizer, input_ids=tk['input_ids'])

len(tk): 3

tk.keys(): dict_keys(['input_ids', 'attention_mask', 'length'])

Number of Tokens: 6

input_ids:
[2, 106, 1645, 108, 107, 108]


tokenizer.convert_ids_to_tokens(input_ids):
['<bos>', '<start_of_turn>', 'user', '\n', '<end_of_turn>', '\n']


tokenizer.decode(input_ids):
<bos><start_of_turn>user
<end_of_turn>




In [8]:
# Tokenizer the entire text
message = [{'role': 'user', 'content': text}]
tk = tokenizer.apply_chat_template(conversation=message,
                                   tokenize=True,
                                   add_generation_prompt=False,
                                   return_dict=True,
                                   tokenizer_kwargs={'return_length': True}
                                   )

# Number of tokens in the text
total_tokens_text = tk["length"][0]

print(f'len(tk): {len(tk)}\n')
print(f'tk.keys(): {tk.keys()}\n')
print(f'Number of Tokens: {total_tokens_text}\n')
view_tokens(tok=tokenizer, input_ids=tk['input_ids'])

len(tk): 3

tk.keys(): dict_keys(['input_ids', 'attention_mask', 'length'])

Number of Tokens: 21

input_ids:
[2, 106, 1645, 108, 1596, 603, 2793, 674, 877, 614, 788, 129632, 675, 40768, 604, 3890, 3287, 1281, 4381, 107, 108]


tokenizer.convert_ids_to_tokens(input_ids):
['<bos>', '<start_of_turn>', 'user', '\n', 'This', '▁is', '▁text', '▁that', '▁will', '▁be', '▁ch', 'unked', '▁with', '▁overlap', '▁for', '▁simple', '▁example', '▁use', '▁cases', '<end_of_turn>', '\n']


tokenizer.decode(input_ids):
<bos><start_of_turn>user
This is text that will be chunked with overlap for simple example use cases<end_of_turn>




<h1 id="approach-1-right-side-truncation">Approach 1: Right Side Truncation</h1>


This apporach takes the first portion of text (from 0 to max_length) and is the most common approach found in tokenizing text.

##### [Return To Top](#intro)

In [9]:
# Keep the right side of the text
message = [{'role': 'user', 'content': text}]
tk = tokenizer.apply_chat_template(conversation=message,
                                   tokenize=True,
                                   add_generation_prompt=False,
                                   return_dict=True,
                                   max_length=max_length,
                                   truncation=True,
                                   tokenizer_kwargs={'return_length': True}
                                   )
print(f'Number of Tokens: {tk["length"][0]}\n')
view_tokens(tok=tokenizer, input_ids=tk['input_ids'])

Number of Tokens: 8

input_ids:
[2, 106, 1645, 108, 1596, 603, 2793, 674]


tokenizer.convert_ids_to_tokens(input_ids):
['<bos>', '<start_of_turn>', 'user', '\n', 'This', '▁is', '▁text', '▁that']


tokenizer.decode(input_ids):
<bos><start_of_turn>user
This is text that



<h1 id="approach-2-right-left-side-truncation">Approach 2: Right and Left Side Truncation</h1>

In this approach the right and left side of the text will be taken and combined together. This will require two separate tokenizations of the text, recombining it, and then tokenizing in the chat template.

Some math needs to be performed to account for the max_length to take from each side. When doing this account for:
- Number of tokens to take for each side (i.e., max_length/2),
- Number of tokens in the chat template (-1 because it also starts with `<bos>` token),
- The `<bos>` token will be dropped at the start of each side of text.

In the below cell notice the use of `math.floor` and `math.ceil` which are used if an odd `max_length` is set. This will ensure that the returned number of tokens is exactly `max_length`. 

##### [Return To Top](#intro)

In [10]:
import math

# Offset accounts for input_ids[1:] for each side
offset = 1

# Calculate the number of tokens for each side
max_len_right = math.floor(((max_length - (num_token_chat_template - 1)) / 2)) + offset
max_len_left = math.ceil(((max_length - (num_token_chat_template - 1)) / 2)) + offset

# Actual length b/c dropping <bos> on each side (i.e., input_ids[1:])
actual_right = max_len_right - 1
actual_left = max_len_left - 1

print(f'user specified -> max_length: {max_length}')
print(f'max_len_right: {max_len_right}')
print(f'max_len_left: {max_len_left}')
print(f'actual_right: {actual_right}')
print(f'actual_left: {actual_left}')
print(f'Number of tokens: {actual_right + actual_left + (num_token_chat_template - 1)}')

user specified -> max_length: 8
max_len_right: 2
max_len_left: 3
actual_right: 1
actual_left: 2
Number of tokens: 8


In [11]:
# Right side (same as approach 1 -> default)
tk_right = tokenizer(text,
                     max_length=max_len_right,
                     truncation=True,
                     return_length=True,
                     padding=False)

# Left side
tokenizer.truncation_side = 'left'
tk_left = tokenizer(text,
                    max_length=max_len_left,
                    truncation=True,
                    return_length=True,
                    padding=False)

# Set truncation_side back to the default "right"
tokenizer.truncation_side = 'right'

# Combine right and left input_ids
combine_input_ids = tk_right['input_ids'][1:] + tk_left['input_ids'][1:]
combine_text = tokenizer.decode(combine_input_ids)

# Tokenizer
message = [{'role': 'user', 'content': combine_text}]
tk = tokenizer.apply_chat_template(conversation=message,
                                   tokenize=True,
                                   add_generation_prompt=False,
                                   return_dict=True,
                                   tokenizer_kwargs={'return_length': True}
                                   )
print(f'tk_left["length"][0] - 1: {tk_left["length"][0] - 1}')
print(f'tk_right["length"][0] - 1: {tk_right["length"][0] - 1}')
print(f'len(combine_input_ids): {len(combine_input_ids)}')
print(f'Number of Tokens: {tk["length"][0]}\n')
view_tokens(tok=tokenizer, input_ids=tk['input_ids'])

tk_left["length"][0] - 1: 2
tk_right["length"][0] - 1: 1
len(combine_input_ids): 3
Number of Tokens: 9

input_ids:
[2, 106, 1645, 108, 1596, 1281, 4381, 107, 108]


tokenizer.convert_ids_to_tokens(input_ids):
['<bos>', '<start_of_turn>', 'user', '\n', 'This', '▁use', '▁cases', '<end_of_turn>', '\n']


tokenizer.decode(input_ids):
<bos><start_of_turn>user
This use cases<end_of_turn>




<h1 id="approach-3-chunking-overlap">Approach 3: Chunking with Overlap</h1>

In Hugging Face tokenizers, the return_overflowing_tokens and stride arguments are used together to handle long texts by splitting them into smaller chunks that the model can process.

`return_overflowing_tokens`: When set to True, this argument instructs the tokenizer to return tokens that "overflow" beyond the specified max_length. This means if a text is too long to fit within the max_length, it will be split into multiple chunks, and each chunk will be returned as part of the output.

`stride`: This argument defines the number of tokens to overlap between consecutive chunks. For example, if stride is set to 5 and max_length is 10, the tokenizer will create chunks that overlap by 5 tokens. This is useful for maintaining context across chunks, especially in tasks like question answering, where understanding context is critical.

Together, these arguments allow for efficient processing of long texts by creating overlapping chunks, ensuring that important contextual information is preserved across the boundaries of the chunks.

##### [Return To Top](#intro)

In [12]:
# Total number of tokens / max_length
print(f'Number of chunks not accounting for stride: '
      f'{total_tokens_text / max_length}')

Number of chunks not accounting for stride: 2.625


In [13]:
# Tokenizer
message = [{'role': 'user', 'content': text}]
tk = tokenizer.apply_chat_template(conversation=message,
                                   tokenize=True,
                                   add_generation_prompt=False,
                                   return_dict=True,
                                   max_length=max_length,
                                   truncation=True,
                                   padding=False,
                                   tokenizer_kwargs={'return_length': True,
                                                     'stride': stride,
                                                     'return_overflowing_tokens': True}
                                   )
print(f'Number of Chunks: {len(tk["input_ids"])}')
for ii, input_ids in enumerate(tk['input_ids']):
    print(f'{ii + 1} of {len(tk["input_ids"])}: {tk["input_ids"][ii][:5]}')

Number of Chunks: 3
1 of 3: [2, 106, 1645, 108, 1596]
2 of 3: [674, 877, 614, 788, 129632]
3 of 3: [604, 3890, 3287, 1281, 4381]


In the above cell only the first chunk has a `<bos>` token which means that when using stride in the `apply_chat_template` it will chunk the entire chat template. Let's observe how tokenizer with stride chunks the text.

In [14]:
# Tokenizer with stride
tk = tokenizer(text,
               max_length=max_length,
               stride=stride,
               truncation=True,
               return_overflowing_tokens=True,
               return_length=True,
               padding=False,
               )
print(f'Number of Chunks: {len(tk["input_ids"])}')
for ii, input_ids in enumerate(tk['input_ids']):
    print(f'{ii + 1} of {len(tk["input_ids"])} ({tk["length"][ii]}): {tk["input_ids"][ii][:5]}')

Number of Chunks: 3
1 of 3 (8): [2, 1596, 603, 2793, 674]
2 of 3 (8): [2, 788, 129632, 675, 40768]
3 of 3 (4): [2, 3287, 1281, 4381]


The tokenizer returns the `<bos>` token at the start of each chunk. To incorporate each chunk of text into the chat template a custom approach will be applied.

In [15]:
# Tokenizer with stride
tk = tokenizer(text,
               max_length=max_length - (num_token_chat_template - 1),
               stride=stride,
               truncation=True,
               return_overflowing_tokens=True,
               return_length=True,
               padding=False,
               )

results = {'input_ids': [],
           'attention_mask': [],
           'length': []}
for ii, input_id in enumerate(tk['input_ids']):
    message = [{'role': 'user',
                'content': tokenizer.decode(input_id[1:])}]
    tk_chat = tokenizer.apply_chat_template(
        conversation=message,
        tokenize=True,
        add_generation_prompt=False,
        return_dict=True,
        tokenizer_kwargs={'return_length': True})
    for field in ['input_ids', 'attention_mask', 'length']:
        results[field] += [tk_chat[field]]

In [16]:
# First chunk of data
print('FIRST CHUNK OF DATA')
print(f'Number of Tokens: {results["length"][0]}\n')
view_tokens(tok=tokenizer, input_ids=results['input_ids'][0])

FIRST CHUNK OF DATA
Number of Tokens: [8]

input_ids:
[2, 106, 1645, 108, 1596, 603, 107, 108]


tokenizer.convert_ids_to_tokens(input_ids):
['<bos>', '<start_of_turn>', 'user', '\n', 'This', '▁is', '<end_of_turn>', '\n']


tokenizer.decode(input_ids):
<bos><start_of_turn>user
This is<end_of_turn>




In [17]:
# Last chunk of data
print('LAST CHUNK OF DATA')
print(f'Number of Tokens: {results["length"][-1]}\n')
view_tokens(tok=tokenizer, input_ids=results['input_ids'][-1])

LAST CHUNK OF DATA
Number of Tokens: [8]

input_ids:
[2, 106, 1645, 108, 1589, 4381, 107, 108]


tokenizer.convert_ids_to_tokens(input_ids):
['<bos>', '<start_of_turn>', 'user', '\n', 'use', '▁cases', '<end_of_turn>', '\n']


tokenizer.decode(input_ids):
<bos><start_of_turn>user
use cases<end_of_turn>




<h1 id="takeaways">Takeaways</h1>

Please feel free to experiment with this notebook and tokenizers. Checkout:
- The approaches in this notebook have been coded into the [/src/preprocess.py](./src/preprocess.py) module. 
- Try out the [dataset-and-collator.ipynb](./dataset-and-collator.ipynb) to see more about the next step in the text processing pipeline which is forming datasets and using collators.

##### [Return To Top](#intro)